In [6]:
import pickle
import os

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [7]:
from sklearn.pipeline import make_pipeline

In [8]:
import mlflow

os.environ["AWS_PROFILE"] = "mac-mlflow" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-3-88-221-2.compute-1.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

#mlflow.set_tracking_uri("http://127.0.0.1:5000")

mlflow.set_experiment("green-taxi-duration")

2024/06/27 13:36:45 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-artifacts-aws/3', creation_time=1719488205912, experiment_id='3', last_update_time=1719488205912, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [9]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [11]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [12]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

/opt/miniconda3/envs/mlflow/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


In [13]:
from mlflow.tracking import MlflowClient


In [14]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '3259f42a340948caa5ec2825221db8b3'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [15]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

/opt/miniconda3/envs/mlflow/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MlflowException: The following failures occurred while downloading one or more artifacts from s3://mlflow-artifacts-aws/3/3259f42a340948caa5ec2825221db8b3/artifacts:
##### File dict_vectorizer.bin #####
An error occurred (404) when calling the HeadObject operation: Not Found

In [22]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [23]:
dv

DictVectorizer()